In [1]:
import pandas as pd
import sqlite3

Dump the smallest tables as-is

In [5]:
con = sqlite3.connect("database.sqlite")
pd.read_sql_query("select * from Country", con).to_csv('Country.csv', index=False)
pd.read_sql_query("select * from League", con).to_csv('League.csv', index=False)
pd.read_sql_query("select * from Team", con).to_csv('Team.csv', index=False)
pd.read_sql_query("select * from Team_Attributes", con).to_csv('Team_Attributes.csv', index=False)
con.close()

Filter out the data on Match and "flatten" the records

In [6]:
# Read the raw data
con = sqlite3.connect("database.sqlite")
df = pd.read_sql_query("select * from Match", con)
df.dropna(inplace=True)
con.close()

tmp = []
for row in df.itertuples():
    # Extract the data we will keep
    match_id = getattr(row, 'id')
    away_team_id = getattr(row, 'away_team_api_id')
    home_team_id = getattr(row, 'home_team_api_id')
    country_id = getattr(row, 'country_id')
    away_players = set([int(getattr(row, f'away_player_{i}')) for i in range(1,12)])
    home_players = set([int(getattr(row, f'home_player_{i}')) for i in range(1,12)])
    # Now create the rows
    for home_player in home_players:
        for away_player in away_players:
            tmp.append([match_id, country_id, away_team_id, home_team_id, away_player, home_player])

df_match = pd.DataFrame(tmp, columns=['match_id', 'country_id', 'away_team_id', 'home_team_id', 'away_player_id', 'home_player_id'])
df_match.to_csv('Match.csv', index=False)
df_match

,match_id,country_id,away_team_id,home_team_id,away_player_id,home_player_id
0,3249,1729,8472,9825,33863,31013
1,3249,1729,8472,9825,175947,31013
2,3249,1729,8472,9825,37868,31013
3,3249,1729,8472,9825,30352,31013
4,3249,1729,8472,9825,25075,31013
...,...,...,...,...,...,...
213197,23417,21518,8634,8370,150739,38367
213198,23417,21518,8634,8370,30613,38367
213199,23417,21518,8634,8370,213688,38367
213200,23417,21518,8634,8370,30909,38367


extract the list of all the players we've seen, that will be used next

In [8]:
players = set(df_match['away_player_id']) | set(df_match['home_player_id'])
len(players)

2344

filter out Player_Attributes to lower its size and only keep stats for active players

In [14]:
# Read the raw data
con = sqlite3.connect("database.sqlite")
df = pd.read_sql_query("select * from Player_Attributes", con)
df.dropna(inplace=True)
con.close()

# Drop all the columns we won't use
df.drop(columns=set(df.columns.values) - set(['player_api_id', 'overall_rating']), inplace=True)

# Keep only players we see in matches
df = df[df['player_api_id'].isin(players)]

# Average the overall rating over time and save the output
df.groupby(by='player_api_id').agg('mean').apply(lambda x: round(round(x) / 2.0) * 2.0).reset_index().to_csv('Player_Attributes.csv', index=False)

finally, we filter the player information to keep only players that play

In [16]:
# Read the raw data
con = sqlite3.connect("database.sqlite")
df = pd.read_sql_query("select * from Player", con)
df.dropna(inplace=True)
con.close()

# Keep only players we see in matches
df = df[df['player_api_id'].isin(players)]

# Save
df.to_csv('Player.csv', index=False)
